In [ ]:
# Land/water mask WIP notebook

# Author: Lee Hathccock

# Bring in our libraries - GDAL and OSR for reading GeoTIFFs, NumPy for array
# manipulation, matplotlib for displaying imagery

import os, sys

from osgeo import gdal
from osgeo import osr
from osgeo import ogr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors


In [ ]:
# Open the image file
#img = gdal.Open("2023-08-24_USDA_NACA_400ft_BGREN_11pTarp.tif")
img = gdal.Open("2023-07-07_USDA_NACA_400ft_BGREN_DLS_11pTarp.tif")

# Get our image's geographic information
geotransform = img.GetGeoTransform()
wkt = img.GetProjection()
proj = osr.SpatialReference(wkt)

xcellsize = geotransform[1]
ycellsize = geotransform[5]

In [ ]:
# Read in our five bands. Note that you may not need all bands, but for
# availability we're going to bring in all of them.

# This will put the raster values for each band into a NumPy array.

blue = img.GetRasterBand(1).ReadAsArray()
green = img.GetRasterBand(2).ReadAsArray()
red = img.GetRasterBand(3).ReadAsArray()
rededge = img.GetRasterBand(4).ReadAsArray()
nir = img.GetRasterBand(5).ReadAsArray()

In [ ]:
# Get our nodata value

# This may actually change based on each band - probably not common, but
# still possible.

nodata = img.GetRasterBand(1).GetNoDataValue()
blue = np.where(blue == nodata, np.nan, blue)
nodata = img.GetRasterBand(2).GetNoDataValue()
green = np.where(green == nodata, np.nan, green)
nodata = img.GetRasterBand(3).GetNoDataValue()
red = np.where(red == nodata, np.nan, red)
nodata = img.GetRasterBand(4).GetNoDataValue()
rededge = np.where(rededge == nodata, np.nan, rededge)
nodata = img.GetRasterBand(5).GetNoDataValue()
nir = np.where(nir == nodata, np.nan, nir)

In [ ]:
# The easiest way to change the display scale for imagery is to change the DPI.
# The larger the number, the more it will slow things down.
plt.rcParams['figure.dpi'] = 300

In [ ]:
# Let's take a look at our data...
imblue = plt.imshow(blue)

# Looks a bit dark! Let's see if we can address that in the next section.

In [ ]:
# Grab our standard deviation and mean. Will need it so we can scale our imagery
# more effectively. Currently using two standard deviations.
stdev = np.nanstd(blue)
mean = np.nanmean(blue)

print(stdev)
print(mean)

blue_min_std = mean - (stdev*2)
blue_max_std = mean + (stdev*2)

# This calls our normalization function. The color map can be changed in the imshow
# call - there are many built-in options, and custom color maps can also be used.
normalize = colors.Normalize(vmin=blue_min_std, vmax=blue_max_std, clip=True)
imblue = plt.imshow(blue, cmap='Blues_r', norm=normalize)

In [ ]:
# Can compute NDVI if we're interesting in looking for vegetation.

# NDVI is (NIR - Red)/(NIR + Red)

ndvi = np.divide((nir - red),(nir + red))
stdev = np.nanstd(ndvi)
mean = np.nanmean(ndvi)

normalize = colors.Normalize(vmin=(mean - (stdev*2)), vmax=(mean + (stdev*2)), clip=True)
imndvi = plt.imshow(ndvi, cmap='Spectral', norm=normalize)

In [ ]:
# Try to reduce resolution.
"""

# We need to create a temporary GeoTIFF file to store our results in. Be careful with this,
# as the rows and columns may be transposed from what the NumPy array holds currently.
driver = gdal.GetDriverByName('GTiff')
outdata = driver.Create('{}'.format("ndvi.tif"), ndvi.shape[1], ndvi.shape[0], 1, \
                        gdal.GDT_Float32, options=['COMPRESS=DEFLATE'])

# Write our modified classified array to the GeoTIFF we just created, and make sure to add
# in our georeferencing as well.
outdata.GetRasterBand(1).WriteArray(ndvi)
outdata.SetGeoTransform(geotransform)
srs = osr.SpatialReference()
srs.ImportFromWkt(wkt)
outdata.SetProjection(srs.ExportToWkt())
outdata.FlushCache()

src_ds = gdal.Open('ndvi.tif')
dst_ds = driver.Create('ndvi_lowres.tif', round(ndvi.shape[1]/5), round(ndvi.shape[0]/5), gdal.GDT_Float32)
dst_ds.SetProjection(srs.ExportToWkt())
dst_ds.SetGeoTransform(geotransform)

#gdal.ReprojectImage(src_ds, dst_ds)
gdal.Warp(dst_ds, src_ds)

dst_ds = None
"""
# We need to create a temporary GeoTIFF file to store our results in. Be careful with this,
# as the rows and columns may be transposed from what the NumPy array holds currently.
driver = gdal.GetDriverByName('GTiff')
outdata = driver.Create('{}'.format("ndvi.tif"), ndvi.shape[1], ndvi.shape[0], 1, \
                        gdal.GDT_Float32, options=['COMPRESS=DEFLATE'])

# Write our modified classified array to the GeoTIFF we just created, and make sure to add
# in our georeferencing as well.
outdata.GetRasterBand(1).WriteArray(ndvi)
outdata.SetGeoTransform(geotransform)
srs = osr.SpatialReference()
srs.ImportFromWkt(wkt)
outdata.SetProjection(srs.ExportToWkt())
outdata.FlushCache()
outdata = None



gdal.Warp("ndvi_lowres.tif", "ndvi.tif", xRes=xcellsize*5, yRes=ycellsize*5, resampleAlg="bilinear")

In [ ]:
# Let's look at NDRE as well. More sensitive to nitrogen content.

# NDRE is (NIR - RedEdge)/(NIR + RedEdge)

ndre = np.divide((nir - rededge),(nir + rededge))
stdev = np.nanstd(ndvi)
mean = np.nanmean(ndvi)

normalize = colors.Normalize(vmin=(mean - (stdev*2)), vmax=(mean + (stdev*2)), clip=True)
imndre = plt.imshow(ndre, cmap='Spectral', norm=normalize)

In [ ]:
# Try to reduce resolution.

# We need to create a temporary GeoTIFF file to store our results in. Be careful with this,
# as the rows and columns may be transposed from what the NumPy array holds currently.
driver = gdal.GetDriverByName('GTiff')
outdata = driver.Create('{}'.format("ndre.tif"), ndre.shape[1], ndre.shape[0], 1, \
                        gdal.GDT_Float32, options=['COMPRESS=DEFLATE'])

# Write our modified classified array to the GeoTIFF we just created, and make sure to add
# in our georeferencing as well.
outdata.GetRasterBand(1).WriteArray(ndre)
outdata.SetGeoTransform(geotransform)
srs = osr.SpatialReference()
srs.ImportFromWkt(wkt)
outdata.SetProjection(srs.ExportToWkt())
outdata.FlushCache()
outdata = None


gdal.Warp("ndre_lowres.tif", "ndre.tif", xRes=xcellsize*5, yRes=ycellsize*5, resampleAlg="bilinear")

In [ ]:
# Using somewhat arbitrary thresholds, let's compute a land/water mask. NDWI defines greater than zero
# as water. We'll also use our NIR band as well to apply a little extra filtering - sometimes NDWI can
# give some false readings, and NIR will help make sure we're not looking at something that's highly
# reflective in the NIR band (possibly in urban environments).

"""
classified = np.where(((ndre >= 0.1) & (ndre < 0.3) & (ndvi > 0)), 1, 2)
classified = np.where(np.isnan(nir), nir, classified)
imclass = plt.imshow(classified, cmap='coolwarm')"""

ndvi_img = gdal.Open('ndvi_lowres.tif')
ndvi = ndvi_img.GetRasterBand(1).ReadAsArray()
ndre_img = gdal.Open('ndre_lowres.tif')
ndre = ndre_img.GetRasterBand(1).ReadAsArray()

conditions  = [ (ndvi <= 0),
                (ndre < 0.1) & (ndvi > 0),
                (ndre >= 0.1) & (ndre < 0.3) & (ndvi > 0), 
                (ndre >= 0.3) & (ndre < 0.4) & (ndvi > 0),
                (ndre >= 0.4) & (ndvi > 0) ]
choices     = [ np.nan, 1, 2, 3, 4 ]
    
classified = np.select(conditions, choices, default=np.nan)
imclass = plt.imshow(classified, cmap='RdYlGn')

In [ ]:
# We're going to try to filter our imagery a bit more, as this is a bit noisy-looking.


# First, we're going to take everywhere that is a NumPy NaN and turn back to zero. This will
# effectively become our nodata value.
classified_nodata = np.where(np.isnan(classified), 0, classified)

# We need to create a temporary GeoTIFF file to store our results in. Be careful with this,
# as the rows and columns may be transposed from what the NumPy array holds currently.
driver = gdal.GetDriverByName('GTiff')
outdata = driver.Create('{}'.format("ndvi_ndre_filter.tif"), classified.shape[1], classified.shape[0], 1, \
                        gdal.GDT_Byte, options=['COMPRESS=DEFLATE'])

# Write our modified classified array to the GeoTIFF we just created, and make sure to add
# in our georeferencing as well.
outdata.GetRasterBand(1).WriteArray(classified_nodata)
outdata.SetGeoTransform(geotransform)
srs = osr.SpatialReference()
srs.ImportFromWkt(wkt)
outdata.SetProjection(srs.ExportToWkt())

# Now we're going to use GDAL's SieveFilter function to group pixels. There are several options to
# change here, but we will stick with a connectedness of 4 (don't consider diagonal pixels adjacent),
# and a threshold value of 50 (groups of pixels smaller than 50 will be disregarded).

gdal.SieveFilter(srcBand=outdata.GetRasterBand(1), maskBand=None, dstBand=outdata.GetRasterBand(1), threshold=50, connectedness=8)
outdata.GetRasterBand(1).SetNoDataValue(0)
outdata.FlushCache()

# Let's take a look at the results!
flt = gdal.Open("ndvi_ndre_filter.tif")
fltimg = flt.GetRasterBand(1).ReadAsArray()
nodata = flt.GetRasterBand(1).GetNoDataValue()
fltimg = np.where(fltimg == nodata, np.nan, fltimg)
imflt = plt.imshow(fltimg, cmap='RdYlGn')


In [ ]:
# Let's filter this a bit more by resampling.
xres = round(fltimg.shape[1] / 5)
yres = round(fltimg.shape[0] / 5)

print(xres, yres)

result = gdal.Warp("", flt, format="VRT", outputType=gdal.GDT_Byte, xRes=xres, yRes=yres)
print (result)